In [ ]:
import os
import sys; sys.path.append('../lib')

import matplotlib.pyplot as plt
import numpy as np; np.random.seed(0)

from data import Cifar
from gradients import compare_gradients
from networks import SingleLayerFullyConnected, TrainHistory

# Constants

In [ ]:
DATA_DIR = '../data'
PICKLE_DIR = '../pickle'

HYPERPARAM_ = {
    'alpha': [0, 0, 0.1, 1],
    'eta': [0.1, 0.01, 0.01, 0.01]
}

# Load dataset

In [ ]:
dataset = Cifar(DATA_DIR)

# Preview data

In [ ]:
dataset.preview()

# Split into training, validation and test set

In [ ]:
data_train, data_val, data_test = \
    dataset.train_val_test_split(n_train=10000, n_val=10000)

# Compare analytical and numerical gradient

In [ ]:
params = [(1, 20, 0), (1, 20, 0.5), (20, 20, 0), (20, 20, 0.5)]

compare_gradients(SingleLayerFullyConnected, data_train, params)

# Train networks

In [ ]:
n_epochs = 40

for alpha, eta in zip(HYPERPARAMS['alpha'], HYPERPARAMS['eta']):
    network = SingleLayerFullyConnected(
        data_train.input_size, data_train.num_classes, alpha=alpha)

    history = network.train(data_train,
                            data_val,
                            n_epochs=n_epochs,
                            verbose=True)

    history.save(PICKLE_DIR, [('alpha', alpha), ('eta', eta)])

In [ ]:
histories = {}
titles = {}

for alpha, eta in zip(HYPERPARAMS['alpha'], HYPERPARAMS['eta']):
    histories[(alpha, eta)] = TrainHistory.load(
        PICKLE_DIR, [('alpha', alpha), ('eta', eta)])

    fmt = r"$\lambda = {}$, $\eta = {}$"
    titles[(alpha, eta)] = fmt.format(alpha, eta)

# Visualize learning curves

In [ ]:
_, axes = plt.subplots(
    NUM_HYPERPARAMS, 2, figsize=(10, 5 * NUM_HYPERPARAMS))

for i, params in enumerate(zip(HYPERPARAMS['alpha'],
                               HYPERPARAMS['eta'])):

    histories[params].visualize(axes=axes[i, :], title=titles[params])

# Analyse network performance

In [ ]:
_, axes = plt.subplots(
    NUM_HYPERPARAMS // 2, 2, figsize=(10, 5 * NUM_HYPERPARAMS // 2))

axes = axes.flatten()

for i, params in enumerate(zip(HYPERPARAMS['alpha'],
                               HYPERPARAMS['eta'])):

    histories[params].final_network.visualize_performance(
        data_test, ax=axes[i], title=titles[params])

# Visualize parameters

In [ ]:
_, axes = plt.subplots(
    NUM_HYPERPARAMS, data_train.num_classes, figsize=(12, 6))

for i, params in enumerate(zip(HYPERPARAMS['alpha'],
                               HYPERPARAMS['eta'])):

    histories[params].final_network.visualize_weights(axes=axes[i, :])

    axes[i, 0].set_ylabel(titles[params], labelpad=50, rotation=0)

for ax, label in zip(axes[-1, :], data_train.labels):
    ax.set_xlabel(label, rotation=45)